# 01c — Performance & Memory (advanced)

Timebox: 45–60 minutes

Goals:
- Identify hotspots and reduce allocations
- Prefer vectorization over Python loops when sensible
- Implement a bounded priority queue



In [ ]:
import heapq
import numpy as np
from typing import Iterable, List

from utils.timers import timer, time_function
from utils.grading import run_checks, assert_true, assert_less


def sum_of_squares_vectorized(x: np.ndarray) -> float:
    return float(np.dot(x, x))


def topk_bounded(iterable: Iterable[int], k: int) -> List[int]:
    if k <= 0:
        return []
    heap = []  # min-heap of size at most k
    for v in iterable:
        if len(heap) < k:
            heapq.heappush(heap, v)
        else:
            if v > heap[0]:
                heapq.heapreplace(heap, v)
    return sorted(heap, reverse=True)

# --- Autograder checks ---
arr = np.arange(100_000, dtype=np.float64)
with timer("sum_of_squares_vectorized"):
    s = sum_of_squares_vectorized(arr)
chk1 = lambda: assert_true("sum nonzero", s > 0)

nums = list(range(1000))
with timer("topk_bounded"):
    tk = topk_bounded(nums, 5)
chk2 = lambda: assert_true("topk correct", tk == [999, 998, 997, 996, 995])

# perf sanity (relative, non-strict)
a = np.random.default_rng(0).normal(size=200_000).astype(np.float64)
loop_ms = time_function(lambda y: float(np.sum(y * y)), a)
vec_ms = time_function(lambda y: sum_of_squares_vectorized(y), a)
chk3 = lambda: assert_true("vectorized faster-ish", vec_ms <= loop_ms * 1.5)

run_checks(chk1, chk2, chk3)

